In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import plotly, pickle
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sn
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [2]:
df=pd.read_csv("loans_full_schema.csv")

In [124]:
df.head(3)

,emp_title,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,annual_income_joint,verification_income_joint,debt_to_income_joint,delinq_2y,months_since_last_delinq,earliest_credit_line,inquiries_last_12m,total_credit_lines,open_credit_lines,total_credit_limit,total_credit_utilized,num_collections_last_12m,num_historical_failed_to_pay,months_since_90d_late,current_accounts_delinq,total_collection_amount_ever,current_installment_accounts,accounts_opened_24m,months_since_last_credit_inquiry,num_satisfactory_accounts,num_accounts_120d_past_due,num_accounts_30d_past_due,num_active_debit_accounts,total_debit_limit,num_total_cc_accounts,num_open_cc_accounts,num_cc_carrying_balance,num_mort_accounts,account_never_delinq_percent,tax_liens,public_record_bankrupt,loan_purpose,application_type,loan_amount,term,interest_rate,installment,grade,sub_grade,issue_month,loan_status,initial_listing_status,disbursement_method,balance,paid_total,paid_principal,paid_interest,paid_late_fees
0,global config engineer,3.0,NJ,MORTGAGE,90000.0,Verified,18.01,NaN,NaN,NaN,0,38.0,2001,6,28,10,70795,38767,0,0,38.0,0,1250,2,5,5.0,10,0.0,0,2,11100,14,8,6,1,92.9,0,0,moving,individual,28000,60,14.07,652.53,C,C3,Mar-2018,Current,whole,Cash,27015.86,1999.33,984.14,1015.19,0.0
1,warehouse office clerk,10.0,HI,RENT,40000.0,Not Verified,5.04,NaN,NaN,NaN,0,NaN,1996,1,30,14,28800,4321,0,1,NaN,0,0,0,11,8.0,14,0.0,0,3,16500,24,14,4,0,100.0,0,1,debt_consolidation,individual,5000,36,12.61,167.54,C,C1,Feb-2018,Current,whole,Cash,4651.37,499.12,348.63,150.49,0.0
2,assembly,3.0,WI,RENT,40000.0,Source Verified,21.15,NaN,NaN,NaN,0,28.0,2006,4,31,10,24193,16000,0,0,28.0,0,432,1,13,7.0,10,0.0,0,3,4300,14,8,6,0,93.5,0,0,other,individual,2000,36,17.09,71.40,D,D1,Feb-2018,Current,fractional,Cash,1824.63,281.80,175.37,106.43,0.0


In [5]:
df.dtypes

emp_title                            object
emp_length                          float64
state                                object
homeownership                        object
annual_income                       float64
verified_income                      object
debt_to_income                      float64
annual_income_joint                 float64
verification_income_joint            object
debt_to_income_joint                float64
delinq_2y                             int64
months_since_last_delinq            float64
earliest_credit_line                  int64
inquiries_last_12m                    int64
total_credit_lines                    int64
open_credit_lines                     int64
total_credit_limit                    int64
total_credit_utilized                 int64
num_collections_last_12m              int64
num_historical_failed_to_pay          int64
months_since_90d_late               float64
current_accounts_delinq               int64
total_collection_amount_ever    

In [6]:
#checking percentage of missing values
(round(df.isnull().sum()*100/df.shape[0]))

emp_title                            8.0
emp_length                           8.0
state                                0.0
homeownership                        0.0
annual_income                        0.0
verified_income                      0.0
debt_to_income                       0.0
annual_income_joint                 85.0
verification_income_joint           85.0
debt_to_income_joint                85.0
delinq_2y                            0.0
months_since_last_delinq            57.0
earliest_credit_line                 0.0
inquiries_last_12m                   0.0
total_credit_lines                   0.0
open_credit_lines                    0.0
total_credit_limit                   0.0
total_credit_utilized                0.0
num_collections_last_12m             0.0
num_historical_failed_to_pay         0.0
months_since_90d_late               77.0
current_accounts_delinq              0.0
total_collection_amount_ever         0.0
current_installment_accounts         0.0
accounts_opened_

In [7]:
#85% non-joint applicants - annual_income_joint, verification_income_joint, debt_to_income_joint
#emp_title and emp_length have 8% missing values but considering those people self-employed
#months_since_last_delinq is null for 57% records
#months_since_90d_late is null for 77% records
#months_since_last_credit_inquiry is null for 13% records
#num_accounts_120d_past_due is null for 3% records

In [8]:
df.tax_liens.head()

0    0
1    0
2    0
3    1
4    0
Name: tax_liens, dtype: int64

In [9]:
#inspecting numeric variables
round(df.describe(percentiles=[.25,.5,.75,.95]))

,emp_length,annual_income,debt_to_income,annual_income_joint,debt_to_income_joint,delinq_2y,months_since_last_delinq,earliest_credit_line,inquiries_last_12m,total_credit_lines,open_credit_lines,total_credit_limit,total_credit_utilized,num_collections_last_12m,num_historical_failed_to_pay,months_since_90d_late,current_accounts_delinq,total_collection_amount_ever,current_installment_accounts,accounts_opened_24m,months_since_last_credit_inquiry,num_satisfactory_accounts,num_accounts_120d_past_due,num_accounts_30d_past_due,num_active_debit_accounts,total_debit_limit,num_total_cc_accounts,num_open_cc_accounts,num_cc_carrying_balance,num_mort_accounts,account_never_delinq_percent,tax_liens,public_record_bankrupt,loan_amount,term,interest_rate,installment,balance,paid_total,paid_principal,paid_interest,paid_late_fees
count,9183.0,10000.0,9976.0,1495.0,1495.0,10000.0,4342.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,2285.0,10000.0,10000.0,10000.0,10000.0,8729.0,10000.0,9682.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
mean,6.0,79222.0,19.0,127915.0,20.0,0.0,37.0,2001.0,2.0,23.0,11.0,183606.0,51049.0,0.0,0.0,46.0,0.0,184.0,3.0,4.0,7.0,11.0,0.0,0.0,4.0,27357.0,13.0,8.0,5.0,1.0,95.0,0.0,0.0,16362.0,43.0,12.0,476.0,14459.0,2494.0,1894.0,600.0,0.0
std,4.0,64734.0,15.0,70168.0,8.0,1.0,22.0,8.0,2.0,12.0,6.0,187633.0,53637.0,0.0,1.0,22.0,0.0,2206.0,3.0,3.0,6.0,6.0,0.0,0.0,2.0,26570.0,8.0,5.0,3.0,2.0,9.0,1.0,0.0,10302.0,11.0,5.0,295.0,9965.0,3958.0,3884.0,517.0,2.0
min,0.0,0.0,0.0,19200.0,0.0,0.0,1.0,1963.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,14.0,0.0,0.0,1000.0,36.0,5.0,31.0,0.0,0.0,0.0,0.0,0.0
25%,2.0,45000.0,11.0,86834.0,14.0,0.0,19.0,1997.0,0.0,14.0,7.0,51594.0,19186.0,0.0,0.0,29.0,0.0,0.0,1.0,2.0,2.0,7.0,0.0,0.0,2.0,10000.0,7.0,5.0,3.0,0.0,93.0,0.0,0.0,8000.0,36.0,9.0,256.0,6679.0,929.0,587.0,222.0,0.0
50%,6.0,65000.0,18.0,113000.0,20.0,0.0,34.0,2003.0,1.0,21.0,10.0,114667.0,36927.0,0.0,0.0,47.0,0.0,0.0,2.0,4.0,6.0,10.0,0.0,0.0,3.0,19500.0,11.0,7.0,5.0,1.0,100.0,0.0,0.0,14500.0,36.0,12.0,398.0,12379.0,1563.0,985.0,446.0,0.0
75%,10.0,95000.0,25.0,151546.0,26.0,0.0,53.0,2006.0,3.0,29.0,14.0,267550.0,65421.0,0.0,0.0,63.0,0.0,0.0,3.0,6.0,11.0,14.0,0.0,0.0,5.0,36100.0,17.0,10.0,7.0,2.0,100.0,0.0,0.0,24000.0,60.0,15.0,645.0,20690.0,2616.0,1695.0,825.0,0.0
95%,10.0,175000.0,37.0,240000.0,34.0,1.0,75.0,2012.0,7.0,45.0,23.0,532598.0,141890.0,0.0,1.0,78.0,0.0,725.0,8.0,10.0,19.0,23.0,0.0,0.0,8.0,78804.0,28.0,18.0,12.0,5.0,100.0,0.0,1.0,36000.0,60.0,21.0,1096.0,34107.0,5920.0,4808.0,1644.0,0.0
max,10.0,2300000.0,469.0,1100000.0,40.0,13.0,118.0,2015.0,29.0,87.0,51.0,3386034.0,942456.0,3.0,52.0,128.0,1.0,199308.0,35.0,29.0,24.0,51.0,0.0,1.0,32.0,386700.0,66.0,46.0,43.0,14.0,100.0,52.0,3.0,40000.0,60.0,31.0,1567.0,40000.0,41630.0,40000.0,4216.0,53.0


In [10]:
# some people have very high debt_to_income ratio.

#85 loan applicants info shows no deliquency in past 2 years but not as per month_since_last_deliq attribute 
print(df.loc[(df.delinq_2y==0)&(df.months_since_last_delinq<=24)].shape[0])



85


In [11]:
df[df.accounts_opened_24m>20].shape

(11, 55)

In [12]:
df.shape

(10000, 55)

In [13]:
#attributes with more than 30 unique values
print("Following attributes have more than 30 unique values:")
for attribute in df.columns:
    if df[attribute].dtype=='object' and df[attribute].nunique()>30:
        print('-----------------------------------')
        print(attribute)
        print('Unique values:',df[attribute].nunique())
#         print(df[attribute].value_counts())


Following attributes have more than 30 unique values:
-----------------------------------
emp_title
Unique values: 4741
-----------------------------------
state
Unique values: 50
-----------------------------------
sub_grade
Unique values: 32


In [14]:
#Relative frequency within attributes with <= 30 unique values
for attribute in df.columns:
    if df[attribute].dtype=='object' and df[attribute].nunique()<=30:
        print('-----------------------------------')
        print(attribute)
        print('Unique values:',df[attribute].nunique())
        print(round(df[attribute].value_counts(normalize=True),2)*100)

-----------------------------------
homeownership
Unique values: 3
MORTGAGE    48.0
RENT        39.0
OWN         14.0
Name: homeownership, dtype: float64
-----------------------------------
verified_income
Unique values: 3
Source Verified    41.0
Not Verified       36.0
Verified           23.0
Name: verified_income, dtype: float64
-----------------------------------
verification_income_joint
Unique values: 3
Not Verified       42.0
Source Verified    34.0
Verified           24.0
Name: verification_income_joint, dtype: float64
-----------------------------------
loan_purpose
Unique values: 12
debt_consolidation    51.0
credit_card           22.0
other                  9.0
home_improvement       7.0
major_purchase         3.0
medical                2.0
house                  2.0
car                    1.0
small_business         1.0
moving                 1.0
vacation               1.0
renewable_energy       0.0
Name: loan_purpose, dtype: float64
-----------------------------------
applic

In [15]:
#Checking if we have analyzed all columns
if (len([x for x in df.columns if df[x].dtype=='object'])+df.describe().shape[1]==df.shape[1]):
    print("Yes we have analyzed all columns!!!")
else:
    print('categorical columns analyzed - ',len([x for x in df.columns if df[x].dtype=='object']))
    print('numeric columns analyzed - ',df.describe().shape[1])
    print("total columns")

Yes we have analyzed all columns!!!


In [16]:
# Thus the only issue in the dataset is of missing values in following columns:

#85% non-joint applicants - annual_income_joint, verification_income_joint, debt_to_income_joint
#emp_title and emp_length have 8% missing values but considering those people self-employed
#months_since_last_delinq is null for 57% records
#months_since_90d_late is null for 77% records
#months_since_last_credit_inquiry is null for 13% records
#num_accounts_120d_past_due is null for 3% records

# Missing value treatment for lightgbm model

In [3]:
# Imputing missing values
df.loc[df.annual_income_joint.isnull(),'annual_income_joint']=0

In [4]:
df[['verification_income_joint', 'debt_to_income_joint']].head(10)

,verification_income_joint,debt_to_income_joint
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,Verified,37.66
5,NaN,NaN
6,Not Verified,13.12
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [5]:
df.loc[df.debt_to_income_joint.isnull(),'debt_to_income_joint']=-1

In [6]:
df.loc[df.verification_income_joint.isnull(),'verification_income_joint']='Not required'

In [7]:
#emp_title and emp_length have 8% missing values
df.loc[df.emp_title.isnull(),'emp_title']='No information'
df.loc[df.emp_length.isnull(),'emp_length']=df.emp_length.median()

In [8]:
#months_since_last_delinq - 57% null values
#for first iteration of modeling, we will drop this variable and impute later for using if necessary 
df=df.drop('months_since_last_delinq',axis=1)

In [9]:
#months_since_90d_late is null for 77% records
#for first iteration of modeling, we will drop this variable and impute later for using if necessary 
df=df.drop('months_since_90d_late',axis=1)


In [10]:
# months_since_last_credit_inquiry - 13% null values
print(df[['months_since_last_credit_inquiry']].describe())

       months_since_last_credit_inquiry
count                       8729.000000
mean                           7.340703
std                            5.981590
min                            0.000000
25%                            2.000000
50%                            6.000000
75%                           11.000000
max                           24.000000


In [11]:
# we will impute this with the median value for this first modeling iteration
df.loc[df.months_since_last_credit_inquiry.isnull(),'months_since_last_credit_inquiry']=df.months_since_last_credit_inquiry.median()

In [12]:
#num_accounts_120d_past_due is null for 3% records
# we will impute this with the median value for this first modeling iteration
df.loc[df.num_accounts_120d_past_due.isnull(),'num_accounts_120d_past_due']=df.num_accounts_120d_past_due.median()


In [13]:
#debt_to_income still has some null values
df.isnull().sum()

emp_title                            0
emp_length                           0
state                                0
homeownership                        0
annual_income                        0
verified_income                      0
debt_to_income                      24
annual_income_joint                  0
verification_income_joint            0
debt_to_income_joint                 0
delinq_2y                            0
earliest_credit_line                 0
inquiries_last_12m                   0
total_credit_lines                   0
open_credit_lines                    0
total_credit_limit                   0
total_credit_utilized                0
num_collections_last_12m             0
num_historical_failed_to_pay         0
current_accounts_delinq              0
total_collection_amount_ever         0
current_installment_accounts         0
accounts_opened_24m                  0
months_since_last_credit_inquiry     0
num_satisfactory_accounts            0
num_accounts_120d_past_du

In [14]:
#replace them by median value as there are only 24 of them
df.loc[df.debt_to_income.isnull(),'debt_to_income']=df.debt_to_income.median()

In [15]:
#No missing values in entire dataset
df.isnull().sum().sum()

0

# One-hot encoding

In [16]:
#dropping some variables
df.drop('emp_title',axis=1,inplace=True)

In [17]:
#one hot encoding
print(df.shape)
for x in df.columns:
    if df[x].dtype=='object':
        print(x)
        print('Unique values:',df[x].nunique())
        print('columns in dataframe',df.shape[1])
        mdf=pd.get_dummies(df[x]).iloc[:,:-1]
        cols=mdf.columns.tolist()
        mdf.columns=[x+'_'+s for s in cols]
        
        df=pd.concat([df,mdf],axis=1)

        print('no. of columns added:',mdf.shape[1])
#         print(df.shape,df.shape[1]-df[x].nunique()+1)
        df.drop(x,axis=1,inplace=True)
        print('shape after encoding column',x,'-',df.shape)
        print("-----------------------------------------------------")

(10000, 52)
state
Unique values: 50
columns in dataframe 52
no. of columns added: 49
shape after encoding column state - (10000, 100)
-----------------------------------------------------
homeownership
Unique values: 3
columns in dataframe 100
no. of columns added: 2
shape after encoding column homeownership - (10000, 101)
-----------------------------------------------------
verified_income
Unique values: 3
columns in dataframe 101
no. of columns added: 2
shape after encoding column verified_income - (10000, 102)
-----------------------------------------------------
verification_income_joint
Unique values: 4
columns in dataframe 102
no. of columns added: 3
shape after encoding column verification_income_joint - (10000, 104)
-----------------------------------------------------
loan_purpose
Unique values: 12
columns in dataframe 104
no. of columns added: 11
shape after encoding column loan_purpose - (10000, 114)
-----------------------------------------------------
application_type
Uni

# Preparing the final training and testing sets

In [18]:
#separate independent and dependent variables
print(df.shape)
y=df['interest_rate']
X=df.drop('interest_rate',axis=1)
X.shape

(10000, 154)


(10000, 153)

In [19]:
#train test splitting
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15,random_state=0)

# Cross-validation and training using lightgbm model

In [21]:
#lightgbm 
clf = lgb.LGBMClassifier()

In [22]:
# convert data to lightgbm Dataset object
d_train = lgb.Dataset(X_train, label=y_train)

In [ ]:
#Hyperparameter tuning and Cross-validation

#append results of iterations in this dataframe
tuningDF=pd.DataFrame()

max_depthList=[6,8,10,12,14,16,18,20,22,24,26,28,30]
num_boost_roundList=[50,100,200,300]

for depth in max_depthList:
    for rounds in num_boost_roundList:
        #defining parameters
        SEARCH_PARAMS = {'learning_rate': 0.1,
                        'max_depth': depth,
                        'num_leaves': 32,
                        'feature_fraction': 0.7,
                        'subsample': 0.7}

        FIXED_PARAMS={'objective': 'regression',
                     'metric': 'rmse',
                     'bagging_freq':5,
                     'boosting':'dart',
                     'num_boost_round':rounds,
                     'early_stopping_rounds':10}

        params = {'metric':FIXED_PARAMS['metric'],
                     'objective':FIXED_PARAMS['objective'],
                     **SEARCH_PARAMS}

        # crossvalidation
        cv_results=lgb.cv(params=params,train_set=d_train,stratified=False,verbose_eval=0)

        clf=lgb.train(params=params,train_set=d_train)
        #make predictions
        y_pred=clf.predict(X_test)
        rms = mean_squared_error(y_test, y_pred, squared=False)

        print({'Best num_boost_rounds':len(cv_results['rmse-mean']),
                        'Best RMSE score:': cv_results['rmse-mean'][-1],'Test RMSE':rms,
                        'depth':depth,'rounds':rounds})
        
        #appending results for each iteration
        tuningDF=tuningDF.append({'Best num_boost_rounds':len(cv_results['rmse-mean']),
                        'Best RMSE score:': cv_results['rmse-mean'][-1],'Test RMSE':rms,
                        'depth':depth,'rounds':rounds},ignore_index=True)

In [66]:
tuningDF.sort_values('Test RMSE')


,Best num_boost_rounds,Best RMSE score:,Test RMSE,depth,rounds
19,100.0,0.328695,0.209684,14.0,300.0
18,100.0,0.328695,0.209684,14.0,200.0
17,100.0,0.328695,0.209684,14.0,100.0
16,100.0,0.328695,0.209684,14.0,50.0
25,100.0,0.326552,0.211838,18.0,100.0
26,100.0,0.326552,0.211838,18.0,200.0
24,100.0,0.326552,0.211838,18.0,50.0
27,100.0,0.326552,0.211838,18.0,300.0
30,100.0,0.328452,0.211936,20.0,200.0
31,100.0,0.328452,0.211936,20.0,300.0


In [68]:
#Thus the best max_depth is 14 and best number of boosting rounds is 100

SEARCH_PARAMS = {'learning_rate': 0.1,
                        'max_depth': 14,
                        'num_leaves': 32,
                        'feature_fraction': 0.7,
                        'subsample': 0.7}

FIXED_PARAMS={'objective': 'regression',
             'metric': 'rmse',
             'bagging_freq':5,
             'boosting':'dart',
             'num_boost_round':100,
             'early_stopping_rounds':10}

params = {'metric':FIXED_PARAMS['metric'],
             'objective':FIXED_PARAMS['objective'],
             **SEARCH_PARAMS}

clf=lgb.train(params=params,train_set=d_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4430
[LightGBM] [Info] Number of data points in the train set: 8500, number of used features: 140
[LightGBM] [Info] Start training from score 12.423573


In [69]:
#make predictions
y_pred=clf.predict(X_test)

In [71]:
#RMSE on test set
rms = mean_squared_error(y_test, y_pred, squared=False)
rms

0.20968447208569216

In [72]:
with open('lightgbm_final_model.pkl','wb') as f:
    pickle.dump(clf,f)

In [3]:
with open('lightgbm_final_model.pkl','rb') as f:
    clf=pickle.load(f)

In [4]:
#percentage feature importances
featureImp=pd.DataFrame(clf.feature_importance()/clf.feature_importance().sum(),columns=['importance'])


In [ ]:
featureImp['feature']=X.columns

In [ ]:
# Sorted by feature importance
featureImp=featureImp.sort_values('importance',ascending=False)

In [ ]:
# Next step - Feature selection & engineering for 2nd training iteration